In [1]:
import sys
print(sys.executable) 

c:\Users\Dell\AppData\Local\Programs\Python\Python310\python.exe


In [2]:
import csv 

with open("training.1600000.processed.noemoticon.csv", 'r') as f:
    
    reader = csv.reader(f)

In [1]:
import pandas as pd 



In [2]:
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='latin-1')

In [3]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [4]:
df.columns = ["Sentiment","ID","Date", "Query","User","Tweet"]

In [5]:
df.head()

,Sentiment,ID,Date,Query,User,Tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [6]:
df = df[["Sentiment", "Tweet"]]
df.head()

,Sentiment,Tweet
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk 

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
def clean_sentence(sentence):
    # Convert to lowercase
    sentence = sentence.lower()
    
    # Tokenize the sentence
    tokens = word_tokenize(sentence)
    
    # Remove punctuation
    tokens = [word for word in tokens if word.isalnum()]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens


In [9]:
df["Tweet"] = df["Tweet"].apply(clean_sentence)

In [10]:
df.head()

,Sentiment,Tweet
0,0,"[upset, ca, update, facebook, texting, might, ..."
1,0,"[kenichan, dived, many, time, ball, managed, s..."
2,0,"[whole, body, feel, itchy, like, fire]"
3,0,"[nationwideclass, behaving, mad, ca, see]"
4,0,"[kwesidei, whole, crew]"


In [11]:
# Build a vocabulary
from collections import Counter

def build_vocab(sentences):
    all_tokens = [token for sentence in sentences for token in sentence]
    token_counts = Counter(all_tokens)
    vocab = {token: idx + 1 for idx, (token, _) in enumerate(token_counts.items())}
    vocab['<PAD>'] = 0
    return vocab

vocab = build_vocab(sentences= df["Tweet"])

print(f"Vocabulary length: {len(vocab)}")

Vocabulary length: 562182


In [12]:
# Convert sentences to sequences of indices
def sentence_to_indices(sentence, vocab):
    return [vocab[token] for token in sentence]

df["Tensor_Tweets"] = df["Tweet"].apply(lambda x : sentence_to_indices(x, vocab))

In [13]:
df.head()

,Sentiment,Tweet,Tensor_Tweets
0,0,"[upset, ca, update, facebook, texting, might, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
1,0,"[kenichan, dived, many, time, ball, managed, s...","[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]"
2,0,"[whole, body, feel, itchy, like, fire]","[24, 25, 26, 27, 28, 29]"
3,0,"[nationwideclass, behaving, mad, ca, see]","[30, 31, 32, 2, 33]"
4,0,"[kwesidei, whole, crew]","[34, 24, 35]"


In [14]:

from sklearn.model_selection import train_test_split

X,Y = df["Tensor_Tweets"].values, df["Sentiment"].values 

X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size = 0.15)

print(f'Shape of training sample is {X_train.shape}')
print(f'Shape of testing sample  is {X_test.shape}')

Shape of training sample is (1359999,)
Shape of testing sample  is (240000,)


In [15]:
MAX_SENTENCE_LENGTH = 20

In [16]:
from torch.nn.utils.rnn import pad_sequence
import torch 

def pad_sentences(sentences):
    sequences = [torch.tensor(sentence) for sentence in sentences] 
    return pad_sequence(sequences, padding_value = 0)

X = pad_sentences(df["Tensor_Tweets"])
X = X.T
print(f"Padded sequence shape {X.shape}")

Padded sequence shape torch.Size([1599999, 50])


In [17]:
Y = torch.tensor(df["Sentiment"], dtype = torch.int64)

Y = Y/4 
Y = Y.int()
print(Y)

tensor([0, 0, 0,  ..., 1, 1, 1], dtype=torch.int32)


In [18]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

dataset = TensorDataset(X, Y)

train_size = int(0.85 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # 20% for testing

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


In [19]:
from torch.utils.data import DataLoader 

In [20]:
import torch

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [22]:
# Create DataLoader for each subset
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


## 1. Using LSTM for Sentiment Analysis

LSTMs are a very popular form of Recurrent Neural Networks that uses a Constant Error Carousel (CEC) to solve the problem of vanishing gradient that is common in the RNNs. By using three types of gates, LSTMs are able to control the reading, writing and updating of the values. Here, we use one architecture of LSTMs to estimate the efficiency on the Twitter Sentiment Analysis. 

In [41]:
import torch.nn as nn 

class SentimentAnalysisLSTM(nn.Module):
    
    def __init__(self,
                 vocab_size, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim, 
                 n_layers = 3, 
                 bidirectional = False, 
                 dropout = 0.3):
        super().__init__()
        
        self.__embed = nn.Embedding(vocab_size, embedding_dim)
        self.__lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, 
                              bidirectional = bidirectional,
                              dropout = dropout, 
                              batch_first = True) 
        
        linear_outputs = hidden_dim
        if bidirectional:
            linear_outputs += hidden_dim
            
        self.__fc = nn.Linear(
            linear_outputs, output_dim
        )
        self.__bidirectional = bidirectional
        
        self.__dropout = nn.Dropout(p = dropout)
    
    def forward(self, x):
        embedded = self.__embed(x)
        
        lstm_output, (hidden, cell) = self.__lstm(embedded) 
        
        if self.__bidirectional:
            hidden = self.__dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else: 
            hidden = self.__dropout(hidden[-1,:,:])
        
        output = self.__fc(hidden) 
        return output
    

In [42]:
LSTMClassifier = SentimentAnalysisLSTM(
    vocab_size = len(vocab),
    embedding_dim = 100, 
    hidden_dim = 256,
    output_dim = 1,
    n_layers = 2,
    bidirectional = True, 
    dropout = 0.3
)

In [43]:
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(LSTMClassifier.parameters(), lr = 0.001)

In [44]:
count = 0 
for p in LSTMClassifier.parameters():
    count += p.numel()
    
print(f"Total number of parameters: {count}")

Total number of parameters: 58528857


In [45]:
LSTMClassifier.to(device)

SentimentAnalysisLSTM(
  (_SentimentAnalysisLSTM__embed): Embedding(562182, 100)
  (_SentimentAnalysisLSTM__lstm): LSTM(100, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (_SentimentAnalysisLSTM__fc): Linear(in_features=512, out_features=1, bias=True)
  (_SentimentAnalysisLSTM__dropout): Dropout(p=0.3, inplace=False)
)

In [46]:
EPOCHS = 10

In [47]:
from tqdm import tqdm 

for epoch in range(EPOCHS):
    print("-------------------------")
    print(f"Epoch ${epoch + 1}")
    print()
    
    LSTMClassifier.train()
    train_loss = 0.0
    
    for batch in tqdm(train_loader):
        text, sentiment = batch 
        text = text.to(device)
        sentiment = sentiment.to(device).float()
        
        output = LSTMClassifier(text).squeeze(1)
        
        training_loss = criterion(output, sentiment)
        
        optimizer.zero_grad()
        training_loss.backward()
        optimizer.step()

        train_loss += training_loss.item()
    
    print(f"\tTraining loss: {train_loss/len(train_loader): .6f}")
    
    LSTMClassifier.eval()
    
    test_loss = 0
    correct_predictions = 0
    
    with torch.no_grad():
        for batch in tqdm(test_loader):
            texts, labels = batch
            texts, labels = texts.to(device), labels.to(device).float()
        
            predictions = LSTMClassifier(texts).squeeze(1)
            loss = criterion(predictions, labels)
            test_loss += loss.item()
        
            predicted_labels = torch.round(torch.sigmoid(predictions))
            correct_predictions += (predicted_labels == labels).sum().item()
    
    print(f'\tTest Loss: {test_loss/len(test_loader):.4f}')
    print(f'\tAccuracy: {correct_predictions / len(test_loader.dataset):.4f}')
        

-------------------------
Epoch $1



100%|██████████| 10625/10625 [12:31<00:00, 14.14it/s]


	Training loss:  0.472296


100%|██████████| 1875/1875 [00:18<00:00, 99.12it/s] 


	Test Loss: 0.4444
	Accuracy: 0.7907
-------------------------
Epoch $2



...

	Test Loss: 0.7134
	Accuracy: 0.7653
-------------------------
Epoch $10



100%|██████████| 10625/10625 [09:27<00:00, 18.72it/s]


	Training loss:  0.142447


100%|██████████| 1875/1875 [00:20<00:00, 92.42it/s]

	Test Loss: 0.7205
	Accuracy: 0.7632


In [48]:
# Print final training and testing accuracy 
LSTMClassifier.eval()
correct_predictions = 0
    
with torch.no_grad():
    for batch in tqdm(test_loader):
        texts, labels = batch
        texts, labels = texts.to(device), labels.to(device).float()
        
        predictions = LSTMClassifier(texts).squeeze(1)
        loss = criterion(predictions, labels)
        
        predicted_labels = torch.round(torch.sigmoid(predictions))
        correct_predictions += (predicted_labels == labels).sum().item()

print(f'\tAccuracy on Test Data : {correct_predictions / len(test_loader.dataset):.4f}')

correct_predictions = 0
    
with torch.no_grad():
    for batch in tqdm(train_loader):
        texts, labels = batch
        texts, labels = texts.to(device), labels.to(device).float()
        
        predictions = LSTMClassifier(texts).squeeze(1)
        loss = criterion(predictions, labels)
        
        predicted_labels = torch.round(torch.sigmoid(predictions))
        correct_predictions += (predicted_labels == labels).sum().item()

print(f'\tAccuracy on Training Data : {correct_predictions / len(train_loader.dataset):.4f}')


100%|██████████| 1875/1875 [00:19<00:00, 94.09it/s]


	Accuracy on Test Data : 0.7632


100%|██████████| 10625/10625 [01:57<00:00, 90.45it/s]

	Accuracy on Training Data : 0.9652


We see that LSTMs achieve an accuracy of 76.32 % on the test data and 96.52% on the training data. This suggests that LSTMs are pretty much able to get the best out of it. Yet we want to improve a bit...

# 2 : Using GRUs

GRUs are a simplified version of LSTMs that employs only two gates : the input and the reset gate. It is used for the instances when we need to minimize the number of parameters as the performance tradeoff between LSTMs and GRUs is very less in most cases, while GRUs tend to provide enhanced speed of computation due to lesser number of variables in it.

In [49]:
import torch.nn as nn 

class SentimentAnalysisGRU(nn.Module):
    def __init__(self, 
                 vocab_size, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim, 
                 n_layers = 3, 
                 bidirectional = True, 
                 dropout = 0.3):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        embedded = self.embedding(x)
        gru_out, hidden = self.gru(embedded)
        
        if self.gru.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
            
        output = self.fc(hidden)
        return output

In [50]:
GRUClassifier = SentimentAnalysisGRU(
    vocab_size = len(vocab),
    embedding_dim = 100, 
    hidden_dim = 256,
    output_dim = 1,
    n_layers = 2,
    bidirectional = True, 
    dropout = 0.3
)

In [51]:
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(GRUClassifier.parameters(), lr = 0.001)

In [52]:
count = 0 
for p in GRUClassifier.parameters():
    count += p.numel()
    
print(f"Total number of parameters: {count}")

Total number of parameters: 57951321


In [53]:
GRUClassifier.to(device)

SentimentAnalysisGRU(
  (embedding): Embedding(562182, 100)
  (gru): GRU(100, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [54]:
EPOCHS = 10

In [55]:
from tqdm import tqdm 

for epoch in range(EPOCHS):
    print("-------------------------")
    print(f"Epoch ${epoch + 1}")
    print()
    
    GRUClassifier.train()
    train_loss = 0.0
    
    for batch in tqdm(train_loader):
        text, sentiment = batch 
        text = text.to(device)
        sentiment = sentiment.to(device).float()
        
        output = GRUClassifier(text).squeeze(1)
        
        training_loss = criterion(output, sentiment)
        
        optimizer.zero_grad()
        training_loss.backward()
        optimizer.step()

        train_loss += training_loss.item()
    
    print(f"\tTraining loss: {train_loss/len(train_loader): .6f}")
    
    LSTMClassifier.eval()
    
    test_loss = 0
    correct_predictions = 0
    
    with torch.no_grad():
        for batch in tqdm(test_loader):
            texts, labels = batch
            texts, labels = texts.to(device), labels.to(device).float()
        
            predictions = GRUClassifier(texts).squeeze(1)
            loss = criterion(predictions, labels)
            test_loss += loss.item()
        
            predicted_labels = torch.round(torch.sigmoid(predictions))
            correct_predictions += (predicted_labels == labels).sum().item()
    
    print(f'\tTest Loss: {test_loss/len(test_loader):.4f}')
    print(f'\tAccuracy: {correct_predictions / len(test_loader.dataset):.4f}')
        

-------------------------
Epoch $1



100%|██████████| 10625/10625 [09:01<00:00, 19.60it/s]


	Training loss:  0.477222


100%|██████████| 1875/1875 [00:22<00:00, 83.58it/s]


	Test Loss: 0.4499
	Accuracy: 0.7868
-------------------------
Epoch $2



...

	Test Loss: 0.5632
	Accuracy: 0.7717
-------------------------
Epoch $10



100%|██████████| 10625/10625 [09:03<00:00, 19.57it/s]


	Training loss:  0.279249


100%|██████████| 1875/1875 [00:21<00:00, 88.59it/s]

	Test Loss: 0.5156
	Accuracy: 0.7685


In [57]:
# Print final training and testing accuracy 
GRUClassifier.eval()
correct_predictions = 0
    
with torch.no_grad():
    for batch in tqdm(test_loader):
        texts, labels = batch
        texts, labels = texts.to(device), labels.to(device).float()
        
        predictions = GRUClassifier(texts).squeeze(1)
        loss = criterion(predictions, labels)
        
        predicted_labels = torch.round(torch.sigmoid(predictions))
        correct_predictions += (predicted_labels == labels).sum().item()

print(f'\tAccuracy on Test Data : {correct_predictions / len(test_loader.dataset):.4f}')

correct_predictions = 0
    
with torch.no_grad():
    for batch in tqdm(train_loader):
        texts, labels = batch
        texts, labels = texts.to(device), labels.to(device).float()
        
        predictions = GRUClassifier(texts).squeeze(1)
        loss = criterion(predictions, labels)
        
        predicted_labels = torch.round(torch.sigmoid(predictions))
        correct_predictions += (predicted_labels == labels).sum().item()

print(f'\tAccuracy on Training Data : {correct_predictions / len(train_loader.dataset):.4f}')

100%|██████████| 1875/1875 [00:21<00:00, 87.20it/s]


	Accuracy on Test Data : 0.7699


100%|██████████| 10625/10625 [01:55<00:00, 91.64it/s] 

	Accuracy on Training Data : 0.8933


We see that GRUs achieve an accuracy of 76.99 % on the test data and 89.33% on the training data. This suggests that GRU performance are better than LSTMs with a difference of over 0.67% in the test data. Also, GRUs have a bit faster training due to lesser parameters and simpler iteration. 

Each iteration of LSTM took around 9 min 20s while each iteration of GRUs was done in approximately 8 min 55s. This indicates a difference of 30s, which is significantly lower for GRUs.

# 3. CNN Network

In [23]:
import torch.nn as nn
import torch.nn.functional as F

class SentimentAnalysisCNN(nn.Module):
    def __init__(self, 
                 vocab_size, 
                 embedding_dim, 
                 num_filters, 
                 filter_sizes, 
                 output_dim = 1, 
                 dropout = 0.3):
        super().__init__()
        
        self.__embedding = nn.Embedding(vocab_size, embedding_dim)
        self.__convs = nn.ModuleList([
            nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(fs, embedding_dim)) 
            for fs in filter_sizes
        ])
        self.__fc = nn.Linear(len(filter_sizes) * num_filters, output_dim)
        self.__dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        embedded = self.__embedding(x).unsqueeze(1)  # Add channel dimension for convolution
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.__convs]  # Convolution
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]  # Max pooling
        cat = self.__dropout(torch.cat(pooled, dim=1))  # Concatenate and apply dropout
        output = self.__fc(cat)  # Fully connected layer
        return output

In [24]:
# Hyperparameters
vocab_size = len(vocab)  # Plus 1 for padding token
embedding_dim = 100
num_filters = 100
filter_sizes = [3, 5, 6, 8]  # Window sizes for convolution
output_dim = 1  # Binary classification
dropout = 0.5

CNNClassifier = SentimentAnalysisCNN(vocab_size, embedding_dim, num_filters, filter_sizes, output_dim, dropout)

In [25]:
count = 0 
for p in CNNClassifier.parameters():
    count += p.numel()
    
print(f"Total number of parameters: {count}")

Total number of parameters: 56338801


In [26]:
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(CNNClassifier.parameters(), lr = 0.001)

In [27]:
CNNClassifier.to(device)

SentimentAnalysisCNN(
  (_SentimentAnalysisCNN__embedding): Embedding(562182, 100)
  (_SentimentAnalysisCNN__convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 100), stride=(1, 1))
  )
  (_SentimentAnalysisCNN__fc): Linear(in_features=300, out_features=1, bias=True)
  (_SentimentAnalysisCNN__dropout): Dropout(p=0.5, inplace=False)
)

In [28]:
EPOCHS = 10

In [30]:
from tqdm import tqdm 

for epoch in range(EPOCHS):
    print("-------------------------")
    print(f"Epoch ${epoch + 1}")
    print()
    
    CNNClassifier.train()
    train_loss = 0.0
    
    for batch in tqdm(train_loader):
        text, sentiment = batch 
        text = text.to(device)
        sentiment = sentiment.to(device).float()
        
        output = CNNClassifier(text).squeeze(1)
        
        training_loss = criterion(output, sentiment)
        
        optimizer.zero_grad()
        training_loss.backward()
        optimizer.step()

        train_loss += training_loss.item()
    
    print(f"\tTraining loss: {train_loss/len(train_loader): .6f}")
    
    CNNClassifier.eval()
    
    test_loss = 0
    correct_predictions = 0
    
    with torch.no_grad():
        for batch in tqdm(test_loader):
            texts, labels = batch
            texts, labels = texts.to(device), labels.to(device).float()
        
            predictions = CNNClassifier(texts).squeeze(1)
            loss = criterion(predictions, labels)
            test_loss += loss.item()
        
            predicted_labels = torch.round(torch.sigmoid(predictions))
            correct_predictions += (predicted_labels == labels).sum().item()
    
    print(f'\tTest Loss: {test_loss/len(test_loader):.4f}')
    print(f'\tAccuracy: {correct_predictions / len(test_loader.dataset):.4f}')
        

-------------------------
Epoch $1



100%|██████████| 10625/10625 [05:27<00:00, 32.43it/s]


	Training loss:  0.449093


100%|██████████| 1875/1875 [00:05<00:00, 356.93it/s]


	Test Loss: 0.4509
	Accuracy: 0.7874
-------------------------
Epoch $2



...

	Test Loss: 0.5898
	Accuracy: 0.7727
-------------------------
Epoch $10



100%|██████████| 10625/10625 [05:30<00:00, 32.16it/s]


	Training loss:  0.270289


100%|██████████| 1875/1875 [00:05<00:00, 354.78it/s]

	Test Loss: 0.6138
	Accuracy: 0.7692


In [31]:
# Print final training and testing accuracy 
CNNClassifier.eval()
correct_predictions = 0
    
with torch.no_grad():
    for batch in tqdm(test_loader):
        texts, labels = batch
        texts, labels = texts.to(device), labels.to(device).float()
        
        predictions = CNNClassifier(texts).squeeze(1)
        loss = criterion(predictions, labels)
        
        predicted_labels = torch.round(torch.sigmoid(predictions))
        correct_predictions += (predicted_labels == labels).sum().item()

print(f'\tAccuracy on Test Data : {correct_predictions / len(test_loader.dataset):.4f}')

correct_predictions = 0
    
with torch.no_grad():
    for batch in tqdm(train_loader):
        texts, labels = batch
        texts, labels = texts.to(device), labels.to(device).float()
        
        predictions = CNNClassifier(texts).squeeze(1)
        loss = criterion(predictions, labels)
        
        predicted_labels = torch.round(torch.sigmoid(predictions))
        correct_predictions += (predicted_labels == labels).sum().item()

print(f'\tAccuracy on Training Data : {correct_predictions / len(train_loader.dataset):.4f}')

100%|██████████| 1875/1875 [00:05<00:00, 357.10it/s]


	Accuracy on Test Data : 0.7692


100%|██████████| 10625/10625 [00:30<00:00, 350.95it/s]

	Accuracy on Training Data : 0.9113


There is only a slight improvement to 76.92% on the test data set yet the accuracy on training data is only 91.13%

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)